In [1]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [ ]:
import os
import getpass

os.environ["NVIDIA_API_KEY"] = getpass.getpass()

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
# from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [3]:
tools = [TavilySearchResults(max_results=1)]

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI assistant, your task is to analyze and interpret images, "
            "considering their historical and cultural significance.",
        ),
        ("user", "Describe the image below"), ("user", "{image}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
print("Prompt:", prompt)

In [ ]:
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("hwchase17/structured-chat-agent")


In [5]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)
# llm = ChatNVIDIA(model="playground_neva_22b")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)
# agent = create_structured_chat_agent(llm, tools, prompt)

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "image": lambda x: x["image"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
import base64

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

# Example usage
image_path = "thumbnail.jpg"
base64_image = encode_image_to_base64(image_path)


In [ ]:
from langchain_core.messages import HumanMessage

msg = llm.invoke(
    [
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": "Describe the image below",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ]
        )
    ]
)

In [ ]:
msg.content

In [11]:
agent_executor.invoke({"image":f"data:image/jpeg;base64,{base64_image}"})



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': '1 validation error for Request\nbody -> tools\n  extra fields not permitted (type=value_error.extra)', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
from langchain_core.messages import HumanMessage

agent_executor.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "Describe the image in this file: "},
                {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
            ]
        )
    ]
)

In [ ]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"input":"Describe the image below\n","image":f"data:image/jpeg;base64,{base64_image}"})
#         "messages": [
#             HumanMessage(
#                 content=[
#                     {
#                         "type": "text",
#                         "text": "Describe the image below,"
#                     },
#                     {
#                         "type": "image_url",
#                         "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
#                     },
#                 ]
#             )
#         ],
#     },
#     {"recursion_limit":150}
# )

In [ ]:
agent_executor.invoke({"input": "What is the capital of France?"})